## Créer l'infrastructure

Terraform est préinstallé dans Cloud Shell. Vous pouvez ainsi immédiatement commencer à créer une infrastructure.

Commencez par créer votre exemple de configuration dans un fichier nommé main.tf. Terraform identifie les fichiers se terminant par .tf ou .tf.json comme des fichiers de configuration et les charge lors de son exécution.

In [ ]:
# 1 -Créez le fichier main.tf

touch main.tf

2- Cliquez sur le bouton Ouvrir l'éditeur dans la barre d'outils de Cloud Shell. Vous pouvez passer de Cloud Shell à l'éditeur de code et vice versa en cliquant sur les icônes Ouvrir l'éditeur et Ouvrir le terminal selon le cas. Vous pouvez également cliquer sur le bouton Ouvrir dans une nouvelle fenêtre pour laisser l'éditeur ouvert dans un onglet distinct.

In [ ]:
'''
3 - Dans l'éditeur, 
ajoutez le contenu ci-dessous au fichier main.tf. 
Veillez à remplacer <PROJECT_ID> par l'ID du projet de l'atelier :
''' 

terraform {
  required_providers {
    google = {
      source = "hashicorp/google"
    }
  }
}
provider "google" {
  version = "3.5.0"
  project = "<PROJECT_ID>"
  region  = "us-central1"
  zone    = "us-central1-c"
}
resource "google_compute_network" "vpc_network" {
  name = "terraform-network"
}

## Bloc Terraform

Le bloc terraform {} est nécessaire pour que Terraform identifie le fournisseur à télécharger à partir du registre Terraform. Dans la configuration ci-dessus, la source du fournisseur google est définie sur hashicorp/google qui est la forme abrégée de registry.terraform.io/hashicorp/google.

Vous pouvez également attribuer une version à chaque fournisseur défini dans le bloc required_providers. L'argument version est facultatif, mais recommandé. Il permet de se limiter à une version spécifique du fournisseur ou à une plage de versions afin d'éviter de télécharger un nouveau fournisseur pouvant contenir des modifications destructives. Si la version n'est pas spécifiée, Terraform téléchargera automatiquement le fournisseur le plus récent lors de l'initialisation.

Pour en savoir plus, consultez la documentation sur les [sources de fournisseurs](https://www.terraform.io/docs/configuration/provider-requirements.html).


## Fournisseurs
Le bloc provider sert à configurer le fournisseur désigné (google, dans ce cas). Un fournisseur est chargé de créer et de gérer les ressources. Plusieurs blocs "provider" peuvent exister si une configuration Terraform gère des ressources provenant de différents fournisseurs.

## Initialisation
La première commande à exécuter pour une nouvelle configuration, ou après avoir récupéré une configuration existante à partir du contrôle des versions, est terraform init. Elle initialise un certain nombre de données et de paramètres locaux qui seront utilisés par les commandes ultérieures.

In [ ]:
'''
Initialisez votre nouvelle configuration Terraform 
en exécutant la commande terraform init dans le répertoire contenant le fichier main.tf :
'''

terraform init

## Créer des ressources

In [ ]:
## 1 -À présent, appliquez votre configuration en exécutant la commande terraform apply :

terraform apply

Le signe + figure à côté de la ressource "google_compute_network" "vpc_network" dans la sortie, ce qui signifie que Terraform va la créer. Cette chaîne est suivie des attributs qui vont être définis. Si la valeur affichée est (known after apply), cela signifie qu'elle ne sera connue qu'une fois la ressource créée.

Si le plan a bien été créé, Terraform s'interrompt et attend votre approbation avant de poursuivre. Si un élément du plan semble incorrect ou hasardeux, vous pouvez abandonner sans risque la procédure à ce stade sans modifier votre infrastructure.

Si la commande terraform apply échoue avec une erreur, lisez le message d'erreur et effectuez la correction requise.

2 - Dans le cas présent, le plan semble acceptable. 

Vous pouvez donc saisir yes lorsque vous êtes invité à confirmer votre choix pour poursuivre la procédure.

L'exécution du plan prend quelques minutes, car Terraform s'assure que le réseau a bien été créé :

In [ ]:
# ...
  Enter a value: yes
google_compute_network.vpc_network: Creating...
google_compute_network.vpc_network: Still creating... [10s elapsed]
google_compute_network.vpc_network: Still creating... [20s elapsed]
google_compute_network.vpc_network: Still creating... [30s elapsed]
google_compute_network.vpc_network: Still creating... [40s elapsed]
google_compute_network.vpc_network: Still creating... [50s elapsed]
google_compute_network.vpc_network: Creation complete after 58s [id=terraform-network]
Apply complete! Resources: 1 added, 0 changed, 0 destroyed.

Terraform a maintenant terminé.
Vous pouvez accéder à Cloud Console pour afficher le réseau que vous avez provisionné.

3 - Dans la console, accédez au menu de navigation, puis sélectionnez Réseau VPC. Vous pouvez constater que le réseau terraform-network a été provisionné.


<img src="https://cdn.qwiklabs.com/ixPpe0919FV8gS8Aekxv2tD8zu7LpVC%2B4yGFQyl%2FJzQ%3D" />

4 - Dans Cloud Shell, exécutez la commande terraform show pour inspecter l'état actuel.

In [ ]:
terraform show

Vous pouvez référencer ces valeurs pour configurer d'autres ressources ou sorties, comme nous le verrons dans la suite de cet atelier.

## Modifier l'infrastructure

Dans la section précédente, vous avez utilisé Terraform pour créer un réseau VPC, qui représente une infrastructure de base. Dans cette section, vous allez apporter des modifications à votre configuration afin de voir comment Terraform les gère.

L'infrastructure évolue constamment et Terraform est conçu pour vous aider à gérer et à appliquer ces modifications. À mesure que vous modifiez les configurations Terraform, l'outil crée un plan d'exécution qui ne modifie que ce qui est nécessaire pour atteindre l'état souhaité.

En utilisant Terraform pour modifier l'infrastructure, vous pouvez procéder au contrôle des versions pour les configurations ainsi que pour l'état afin de savoir comment l'infrastructure évolue au fil du temps.

### Ajouter des ressources

Vous pouvez ajouter des ressources en les incluant dans votre configuration Terraform et en exécutant terraform apply pour les provisionner.



In [ ]:
# 1 - Dans l'éditeur, ajoutez une ressource d'instance de calcul à main.tf :

resource "google_compute_instance" "vm_instance" {
  name         = "terraform-instance"
  machine_type = "f1-micro"
  boot_disk {
    initialize_params {
      image = "debian-cloud/debian-11"
    }
  }
  network_interface {
    network = google_compute_network.vpc_network.name
    access_config {
    }
  }
}

Cette ressource inclut quelques autres arguments. 
Le nom et le type de machine sont des chaînes simples. En revanche, boot_disk et network_interface correspondent à des blocs plus complexes. Vous pouvez découvrir toutes les options disponibles dans la documentation.

Pour cet exemple, votre instance de calcul utilisera un système d'exploitation Debian et sera connectée au réseau VPC créé précédemment. Notez que cette configuration fait référence à la propriété "name" du réseau avec google_compute_network.vpc_network.name, où google_compute_network.vpc_network est l'ID et correspond aux valeurs du bloc qui définit le réseau, et name représente une propriété de cette ressource.

La présence du bloc access_config, même sans aucun argument, garantit que l'instance sera accessible sur Internet.

In [ ]:
# 2 - À présent, exécutez terraform apply pour créer l'instance de calcul :

terraform apply

Là encore, répondez **yes** à l'invite de confirmation.

Il s'agit d'une modification assez simple : vous avez ajouté une ressource "google_compute_instance" nommée "vm_instance" à votre configuration, puis Terraform a créé cette ressource dans Google Cloud.

### Modifier des ressources

Terraform permet non seulement de créer des ressources, mais également de les modifier.

In [ ]:
# 1 - Ajoutez un argument tags à la ressource "vm_instance" pour qu'elle se présente comme suit :

resource "google_compute_instance" "vm_instance" {
  name         = "terraform-instance"
  machine_type = "f1-micro"
  tags         = ["web", "dev"]
  # ...
}

In [ ]:
# 2 -Réexécutez terraform apply pour mettre à jour l'instance.

terraform apply

3 - Le préfixe **~** signifie que Terraform va mettre à jour la ressource sur place. Vous pouvez appliquer cette modification maintenant en répondant yes. Terraform ajoute alors les tags à votre instance.

### Modifications destructives
Une modification destructive nécessite que le fournisseur remplace la ressource existante au lieu de la mettre à jour. Elle se produit généralement lorsque le fournisseur cloud ne peut pas mettre à jour la ressource de la manière décrite par votre configuration.

C'est le cas, par exemple, si vous modifiez l'image disque de votre instance.



In [ ]:
# 1 -Modifiez comme suit le bloc boot_disk dans la ressource vm_instance de votre fichier de configuration :

  boot_disk {
    initialize_params {
      image = "cos-cloud/cos-stable"
    }
  }

In [ ]:
'''
 2 - À présent, réexécutez terraform apply pour voir comment Terraform va appliquer cette modification 
 aux ressources existantes :
'''
terraform apply

Le préfixe -/+ signifie que Terraform va détruire et recréer la ressource au lieu de la mettre à jour sur place. Certains attributs (indiqués par le préfixe ~) peuvent être mis à jour sur place. En revanche, vous avez besoin de recréer une instance pour modifier l'image de son disque de démarrage. Terraform et le fournisseur Google Cloud gèrent ces détails pour vous. Par ailleurs, le plan d'exécution indique clairement les opérations qui vont être effectuées par Terraform.

En outre, le plan d'exécution montre que le remplacement de votre instance est dû aux modifications apportées à l'image disque. En utilisant ces informations, vous pouvez ajuster vos modifications pour éviter les mises à jour entraînant des opérations de destruction/création si elles sont inacceptables dans certains cas.

3 - Là encore, Terraform demande que vous approuviez le plan d'exécution avant de poursuivre la procédure. Répondez yes pour exécuter les étapes prévues.

Comme indiqué par le plan d'exécution, Terraform commence par détruire l'instance existante, puis en crée une autre à la place. Vous pouvez réutiliser terraform show pour afficher les nouvelles valeurs associées à cette instance.


## Détruire l'infrastructure

Vous savez maintenant comment concevoir et modifier une infrastructure. Avant de créer plusieurs ressources et afficher les dépendances associées, vous allez découvrir comment détruire complètement votre infrastructure gérée par Terraform.

La destruction d'une infrastructure est un événement rare dans les environnements de production. Toutefois, si vous utilisez Terraform pour lancer plusieurs environnements (de développement, de test et de préproduction, par exemple), la destruction est souvent une action utile.

Vous pouvez détruire les ressources en utilisant la commande terraform destroy. Elle est semblable à terraform apply, mais se comporte comme si toutes les ressources avaient été supprimées de la configuration.

In [ ]:
# 1 - Testez la commande terraform destroy. Répondez yes pour exécuter ce plan et détruire l'infrastructure :

terraform destroy

Le préfixe - indique que l'instance et le réseau vont être détruits. Comme avec "apply", Terraform affiche son plan d'exécution et attend votre approbation avant d'apporter des modifications.

De la même façon que pour terraform apply, Terraform détermine l'ordre dans lequel les éléments doivent être détruits. Google Cloud ne permet pas de supprimer un réseau VPC s'il contient des ressources. Terraform attend donc que l'instance soit détruite avant de détruire le réseau. Lors de l'exécution des opérations, Terraform crée un graphe de dépendances pour déterminer l'ordre dans lequel elles doivent être réalisées. Dans des cas plus compliqués impliquant plusieurs ressources, Terraform effectue les opérations en parallèle si cette façon de procéder ne présente aucun risque.

## Créer des dépendances de ressources

Dans cette section, vous allez en apprendre davantage sur les dépendances de ressources. Vous découvrirez également comment utiliser les paramètres de ressources pour partager des informations sur l'une d'elles avec d'autres ressources.

Les infrastructures réelles comportent un large éventail de ressources et de types de ressources. Les configurations Terraform peuvent inclure plusieurs ressources et types de ressources, et ces types peuvent même englober plusieurs fournisseurs.

Dans cette section, nous allons nous intéresser à un exemple de base montrant comment configurer plusieurs ressources. Nous verrons également comment utiliser les attributs de ressources pour configurer d'autres ressources.



In [ ]:
# 1 -Recréez votre réseau et votre instance. Lorsque vous répondez yes à l'invite, les ressources sont créées.

terraform apply

### Attribuer une adresse IP statique

In [1]:
'''
1- Vous allez maintenant compléter votre configuration en attribuant une adresse IP statique
à l'instance de VM dans main.tf.
''' 

resource "google_compute_address" "vm_static_ip" {
  name = "terraform-static-ip"
}

SyntaxError: invalid syntax (<ipython-input-1-c4741f1b2bff>, line 1)

La procédure est semblable à celle de l'exemple précédent qui vous a permis d'ajouter une ressource d'instance de VM, sauf que cette fois, 

vous allez créer une ressource de type "google_compute_address". Ce type de ressource attribue une adresse IP réservée à votre projet.

In [ ]:
# 2 - Ensuite, exécutez terraform plan.

terraform plan

In [ ]:
# La commande terraform plan vous permet de voir ce qui va être créé :

$ terraform plan
Refreshing Terraform state in-memory prior to plan...
The refreshed state will be used to calculate this plan, but will not be
persisted to local or remote state storage.
google_compute_network.vpc_network: Refreshing state... [id=terraform-network]
google_compute_instance.vm_instance: Refreshing state... [id=terraform-instance]
------------------------------------------------------------------------
An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  + create
Terraform will perform the following actions:
  # google_compute_address.vm_static_ip will be created
  + resource "google_compute_address" "vm_static_ip" {
      + address            = (known after apply)
      + address_type       = "EXTERNAL"
      + creation_timestamp = (known after apply)
      + id                 = (known after apply)
      + name               = "terraform-static-ip"
      + network_tier       = (known after apply)
      + project            = (known after apply)
      + region             = (known after apply)
      + self_link          = (known after apply)
      + subnetwork         = (known after apply)
      + users              = (known after apply)
    }
Plan: 1 to add, 0 to change, 0 to destroy.
------------------------------------------------------------------------
Note: You didn't specify an "-out" parameter to save this plan, so Terraform can't guarantee that exactly these actions will be performed if
"terraform apply" is subsequently run.

Contrairement à terraform apply, la commande plan ne montre que ce qui va changer et n'applique jamais les modifications directement. Notez que la seule modification apportée jusqu'à présent est l'ajout d'une adresse IP statique. Ensuite, vous devez associer l'adresse IP à votre instance.

In [ ]:
# 3 - Mettez à jour comme suit le bloc de configuration network_interface de votre instance :

  network_interface {
    network = google_compute_network.vpc_network.self_link
    access_config {
      nat_ip = google_compute_address.vm_static_ip.address
    }
  }

le bloc access_config comporte plusieurs arguments facultatifs. Ici, vous allez définir nat_ip en tant qu'adresse IP statique. Lorsque l'outil Terraform lit cette configuration, il exécute les opérations suivantes :

Il vérifie que la ressource vm_static_ip a été créée avant vm_instance.

Il enregistre les propriétés de vm_static_ip dans l'état.

Il définit nat_ip sur la valeur de la propriété vm_static_ip.address.


In [ ]:
# 4 -Réexécutez "terraform plan" en veillant cette fois à enregistrer le plan :

terraform plan -out static_ip

En enregistrant le plan de cette manière, vous êtes sûr de pouvoir l'appliquer à l'identique ultérieurement. Si vous essayez d'appliquer le fichier créé par le plan, Terraform vérifie d'abord que le même ensemble de modifications va être apporté avant d'exécuter le plan.

Dans ce cas, vous constatez que Terraform va créer une ressource google_compute_address et mettre à jour la VM existante pour qu'elle l'utilise.

In [ ]:
# 5 - Exécutez terraform apply "static_ip" pour voir comment Terraform prévoit d'appliquer cette modification :

terraform apply "static_ip"

Comme indiqué ci-dessus, Terraform a créé l'adresse IP statique avant de modifier l'instance de VM. Grâce à l'expression d'interpolation qui transmet l'adresse IP à la configuration de l'interface réseau de l'instance, l'outil Terraform peut déduire une dépendance et sait qu'il doit créer l'adresse IP statique avant de mettre à jour l'instance.

## Dépendances implicites et explicites

En étudiant les attributs de ressources utilisés dans les expressions d'interpolation, Terraform peut automatiquement déterminer quand une ressource dépend d'une autre. Dans l'exemple ci-dessus, la référence à google_compute_address.vm_static_ip.address crée une dépendance implicite avec la ressource google_compute_address nommée vm_static_ip.

Terraform utilise ces informations sur les dépendances pour déterminer l'ordre dans lequel créer et mettre à jour différentes ressources. Dans l'exemple ci-dessus, Terraform sait que la ressource vm_static_ip doit être créée avant que la ressource vm_instance soit mise à jour pour s'en servir.

Les dépendances implicites via les expressions d'interpolation constituent le principal moyen d'informer Terraform de ces relations et doivent être utilisées autant que possible.

Dans certains cas, Terraform ne peut pas voir les dépendances entre les ressources. L'argument depends_on peut être ajouté à n'importe quelle ressource et accepte une liste de ressources pour lesquelles des dépendances explicites peuvent être créées.

Par exemple, une application que vous allez exécuter sur votre instance peut s'attendre à utiliser un bucket Cloud Storage spécifique. Toutefois, cette dépendance est configurée dans le code de l'application et Terraform ne peut donc pas la voir. Dans ce cas, vous pouvez utiliser depends_on pour déclarer explicitement la dépendance.

In [ ]:
'''
1 - Ajoutez un bucket Cloud Storage et une instance ayant une dépendance explicite 
avec ce bucket en incluant le code suivant dans main.tf :
'''

# New resource for the storage bucket our application will use.
resource "google_storage_bucket" "example_bucket" {
  name     = "<UNIQUE-BUCKET-NAME>"
  location = "US"
  website {
    main_page_suffix = "index.html"
    not_found_page   = "404.html"
  }
}
# Create a new instance that uses the bucket
resource "google_compute_instance" "another_instance" {
  # Tells Terraform that this VM instance must be created only after the
  # storage bucket has been created.
  depends_on = [google_storage_bucket.example_bucket]
  name         = "terraform-instance-2"
  machine_type = "f1-micro"
  boot_disk {
    initialize_params {
      image = "cos-cloud/cos-stable"
    }
  }
  network_interface {
    network = google_compute_network.vpc_network.self_link
    access_config {
    }
  }
}

*Les buckets de stockage doivent être uniques. Vous devez donc remplacer UNIQUE-BUCKET-NAME par un nom unique et valide pour un bucket. Nous vous recommandons d'utiliser votre nom et la date. Il s'agit généralement d'un bon moyen de créer un nom de bucket unique.*

Vous vous demandez peut-être où ces ressources doivent être placées dans votre configuration. L'ordre dans lequel elles sont définies dans un fichier de configuration Terraform n'a aucune incidence sur la façon dont cet outil applique vos modifications. Organisez vos fichiers de configuration de la manière la plus logique pour vous et votre équipe.


In [ ]:
# 2 - À présent, exécutez "terraform plan" et "terraform apply" pour observer l'effet de ces modifications :

terraform plan
terraform apply

3 - Avant de continuer, supprimez ces nouvelles ressources de votre configuration et réexécutez terraform apply pour les détruire. Vous n'utiliserez plus le bucket ni la deuxième instance dans le guide de démarrage.

## Provisionner l'infrastructure

L'instance de calcul que vous avez lancée à ce stade est basée sur l'image Google fournie. Aucun logiciel supplémentaire n'a été installé et aucune configuration n'a été appliquée.

Google Cloud permet aux clients de gérer leurs propres images personnalisées de système d'exploitation. C'est un excellent moyen de vous assurer que les instances que vous provisionnez avec Terraform sont préconfigurées selon vos besoins. Packer constitue l'outil idéal à cette fin et comprend un compilateur pour Google Cloud.

Terraform utilise des approvisionneurs pour importer des fichiers, exécuter des scripts shell ou installer et déclencher d'autres logiciels, tels que des outils de gestion de la configuration.

### Définir un approvisionneur

In [ ]:
'''
1 -Pour définir un approvisionneur, modifiez le bloc "resource" définissant la première ressource 
vm_instance de votre configuration pour qu'il se présente comme suit :
'''

resource "google_compute_instance" "vm_instance" {
  name         = "terraform-instance"
  machine_type = "f1-micro"
  tags         = ["web", "dev"]
  provisioner "local-exec" {
    command = "echo ${google_compute_instance.vm_instance.name}:  ${google_compute_instance.vm_instance.network_interface[0].access_config[0].nat_ip} >> ip_address.txt"
  }
  # ...
}

Un bloc provisioner est ajouté au bloc resource. Vous pouvez ajouter plusieurs blocs provisioner pour définir différentes étapes de provisionnement. Terraform accepte de [nombreux approvisionneurs](https://www.terraform.io/docs/provisioners/index.html). Toutefois, pour cet exemple, vous allez utiliser l'approvisionneur local-exec.

L'approvisionneur local-exec lance une commande localement sur la machine qui exécute Terraform, et non sur l'instance de VM elle-même. Étant donné que vous avez choisi cet approvisionneur, vous n'avez pas besoin de spécifier des [informations de connexion](https://developer.hashicorp.com/terraform/language/resources/provisioners/connection) pour le moment.

Cet exemple illustre également une interpolation de chaîne plus complexe que celle que nous avons examinée précédemment. Chaque instance de VM peut disposer de plusieurs interfaces réseau. Vous devez donc faire référence à la première avec network_interface[0], en comptant à partir de 0, comme le font la plupart des langages de programmation. Par ailleurs, chaque interface réseau peut inclure plusieurs blocs "access_config" et, là encore, vous devez indiquer le premier.

In [ ]:
# 2 -Exécutez terraform apply. La sortie obtenue à ce stade peut être déroutante au premier abord.

terraform apply

Terraform n'a trouvé aucune opération à exécuter. Si vous effectuez une vérification, vous constatez qu'il n'y a **pas de fichier ip_address.txt sur votre machine locale**.

Terraform traite les approvisionneurs différemment des autres arguments. Les approvisionneurs ne s'exécutent que lorsqu'une ressource est créée, mais l'ajout d'un approvisionneur ne vous oblige pas à détruire et recréer la ressource.

In [ ]:
# 3 - Utilisez terraform taint pour demander à Terraform de recréer l'instance :

terraform taint google_compute_instance.vm_instance

Une ressource marquée comme corrompue avec la commande "taint" sera détruite et recréée la prochaine fois que vous exécuterez apply.

In [ ]:
# 4 -À présent, exécutez terraform apply :

terraform apply

5 - Vérifiez que tout s'est déroulé correctement en consultant le contenu du fichier ip_address.txt.
Il inclut l'adresse IP, comme vous l'avez demandé.

## Approvisionneurs défaillants et ressources marquées comme corrompues

Si une étape de provisionnement échoue pour une ressource qui a bien été créée, Terraform génère une erreur et marque la ressource comme corrompue. Une ressource marquée comme corrompue existe toujours, mais ne peut pas être utilisée sans risque, car le provisionnement a échoué.

Lorsque vous générerez votre prochain plan d'exécution, Terraform supprimera toutes les ressources marquées comme corrompues et en créera d'autres en essayant de les provisionner à nouveau après leur création.

### Approvisionneurs de destruction
Vous pouvez également définir des approvisionneurs qui ne s'exécuteront que lors d'une opération de destruction. Ils sont utiles pour nettoyer le système, extraire des données, etc.

Pour de nombreuses ressources, l'utilisation de mécanismes de nettoyage intégrés (tels que les scripts d'initialisation) est recommandée dans la mesure du possible. Toutefois, vous pouvez vous servir des approvisionneurs si nécessaire.

Cet atelier ne présente pas d'exemple illustrant les approvisionneurs de destruction. Si vous avez besoin de les utiliser, consultez la documentation sur les approvisionneurs.

**🎉Félicitations !🎉**

Dans cet atelier, vous avez appris à créer, modifier et détruire une infrastructure avec Terraform. Vous avez ensuite créé des dépendances de ressources et provisionné l'infrastructure de base avec des fichiers de configuration Terraform.

